In [1]:
# Install necessary libraries if not already installed
!pip install -q transformers datasets tqdm

# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import random

# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-Neo model and tokenizer, and move the model to the device
model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the MedQA dataset with training and testing splits
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
train_dataset = dataset['train']
test_dataset = dataset['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 16.5 MB/s eta 0:00:00
Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [2]:
# Function to create few-shot examples from the training data
def create_few_shot_examples(train_data, num_examples):
    """
    Create a few-shot prompt using num_examples samples from the training dataset.
    Each example will include a question, options, and the correct answer.
    """
    # Shuffle and select random examples from the training set for diversity
    shuffled_data = random.sample(list(train_data), len(train_data))

    examples = ""
    selected_categories = set()  # Keep track of categories to ensure diversity

    # Use tqdm to monitor the progress of few-shot example creation
    for sample in tqdm(shuffled_data, desc="Creating few-shot examples", total=len(train_data)):
        question = sample['question']
        options = sample['options']
        correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

        # Ensure diversity by limiting to different topics or question types (if available)
        if len(selected_categories) >= num_examples:
            break

        # Prepare the few-shot format (Question, Options, Answer)
        options_text = ', '.join([f"{key}: {value}" for key, value in options.items()])
        example = f"Question: {question}\nOptions: {options_text}\nAnswer: {correct_answer}\n\n"
        examples += example

        # Use some logic to track different question types/domains (optional)
        selected_categories.add(question)  # Replace with refined logic for topic detection

    return examples

# Function to perform inference with few-shot prompting and match answer using regex
def get_model_prediction(question, options_dict, few_shot_prompt):
    """
    Generate a few-shot prediction using GPT-Neo for a question with multiple choices.
    The options are provided as a dictionary with keys: 'A', 'B', 'C', 'D'.
    The few-shot examples are passed as part of the prompt.
    """
    # Prepare the prompt with few-shot examples + the current question
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"{few_shot_prompt}\nQuestion: {question}\nOptions: {options_text}\nAnswer:"''

    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (can be tuned using temperature, max_length, etc.)
    output = model.generate(inputs["input_ids"], max_new_tokens=200)

    # Decode the output to text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


    ind=generated_text.find(question)
    extracted_text=generated_text[ind+len(question)+len(options_text)+10:]
    ind2=extracted_text.find("Question")
    predicted=extracted_text[:ind2].strip()

    return predicted

In [ ]:
# Evaluate model on the test dataset with tqdm to monitor the progress
correct_predictions = 0
total_predictions = 0

# Add tqdm to track the evaluation loop
for sample in tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset)):
    few_shot_prompt = create_few_shot_examples(train_dataset, num_examples=3)
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options, few_shot_prompt)

    print(predicted_answer)
    print(correct_answer)

    # Check if prediction is correct
    if correct_answer in predicted_answer:
        correct_predictions += 1

    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {total_predictions}: Accuracy so far: {accuracy:.2f}%")


# Final accuracy
final_accuracy = correct_predictions / total_predictions * 100
print(f"Final Accuracy: {final_accuracy:.2f}%")

Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 25471.48it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.
Evaluating on test dataset:   0%|          | 1/1273 [00:52<18:37:20, 52.70s/it]

Answer: Tell the attending that he cannot fail to disclose this mistake
Tell the attending that he cannot fail to disclose this mistake
Iteration 1: Accuracy so far: 100.00%



Evaluating on test dataset:   0%|          | 2/1273 [01:43<18:11:06, 51.51s/it]

Answer: Inhibition of proteasome
Cross-linking of DNA
Iteration 2: Accuracy so far: 50.00%



Evaluating on test dataset:   0%|          | 3/1273 [02:44<19:43:06, 55.90s/it]

...


Evaluating on test dataset:   7%|▋         | 85/1273 [1:27:03<21:54:15, 66.38s/it]

Answer: Endometrial tissue outside the uterine cavity
Endometrial tissue outside the uterine cavity
Iteration 85: Accuracy so far: 58.82%



Evaluating on test dataset:   7%|▋         | 86/1273 [1:27:59<20:48:14, 63.10s/it]

Answer: Friable irregular masses attached to the valve
Friable irregular masses attached to the valve
Iteration 86: Accuracy so far: 59.30%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 28728.11it/s]


In [ ]:
total_predictions = 86  # Total number of predictions made up to iteration 86
accuracy_at_86 = 59.30  # Accuracy at iteration 86
correct_predictions = int((accuracy_at_86 / 100) * total_predictions)

for i, sample in enumerate(tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset))):
    if i < 86:
        continue

    few_shot_prompt = create_few_shot_examples(train_dataset, num_examples=3)
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options, few_shot_prompt)

    print(predicted_answer)
    print(correct_answer)

    # Check if prediction is correct
    if correct_answer in predicted_answer:
        correct_predictions += 1

    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {i+1}: Accuracy so far: {accuracy:.2f}%")


Evaluating on test dataset:   7%|▋         | 87/1273 [01:02<14:16,  1.38it/s]

Answer: Hysteroscopy
Laparoscopy
Iteration 87: Accuracy so far: 57.47%



Evaluating on test dataset:   7%|▋         | 88/1273 [02:08<35:06,  1.78s/it]

Answer: High LDL-cholesterol
High LDL-cholesterol
Iteration 88: Accuracy so far: 57.95%



Evaluating on test dataset:   7%|▋         | 89/1273 [03:10<1:01:39,  3.12s/it]

...


Evaluating on test dataset:  12%|█▏        | 154/1273 [1:06:44<16:25:29, 52.84s/it]

Answer: Previous suicide attempt
Previous suicide attempt
Iteration 154: Accuracy so far: 61.04%



Evaluating on test dataset:  12%|█▏        | 155/1273 [1:07:42<16:58:44, 54.67s/it]

Answer: Chickenpox
Chickenpox
Iteration 155: Accuracy so far: 61.29%



Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 24291.34it/s]
